# Value Candidates with FMP Stock Screen 
#### Source Article: [Building an Investment Model using Financial Ratios in Python](https://codingandfun.com/building-an-investing-model-using-financial-ratios-and-python/#comments) 
---
## Basic Criteria:

* D/E <= 0.50 low debt 
* Current Ratio > 1.5 more than enough cash to cover current liabilities
* ROE stable growth for the past 10-years
* FCF stable growth last 10 years (cash-rich)
* Operating Margin - stable operating margins over the last 10 years (strong competitive position)
* Companies that grow without issuing additional shares

In [38]:
import numpy as np
import pandas as pd
import datetime as datetime
from datetime import datetime
import requests
import os
import json
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore")

In [39]:
load_dotenv()

True

In [40]:
# Read in API key from env file
FMP_API_KEY = os.getenv('FMP_API_KEY')

# Create Basic Stock Screen

In [41]:
companies = []

market_cap = '1000000000' 
volume = '10000'
beta = 1
dividend = 1
limit = 1000
exchange = 'NYSE,NASDAQ'

url = (f'https://financialmodelingprep.com/api/v3/stock-screener?marketCapMoreThan={market_cap}&betaMoreThan={beta}&volumeMoreThan={volume}&dividendMoreThan={dividend}&exchange={exchange}&limit={limit}&apikey={FMP_API_KEY}')

# Get companies based on criteria defined about
screen = requests.get(url).json()
#screen

# Pull Financial Ratios

In [42]:
# Add selected companies to a list
for item in screen:
	companies.append(item['symbol'])
	
# Print(companies)
value_ratios = {}

# Get the financial ratios
count = 0

for company in companies:

	try:
		if count <100:
			count = count + 1
	
			fin_ratios = requests.get(f'https://financialmodelingprep.com/api/v3/ratios/{company}?apikey={FMP_API_KEY}').json()

			value_ratios[company] = {}

			value_ratios[company]['ROE'] = fin_ratios[0]['returnOnEquity']
			value_ratios[company]['ROA'] = fin_ratios[0]['returnOnAssets']
			value_ratios[company]['Debt_Eq_Ratio'] = fin_ratios[0]['debtEquityRatio']
			value_ratios[company]['Current_Ratio'] = fin_ratios[0]['currentRatio']
			value_ratios[company]['PB'] = fin_ratios[0]['priceToBookRatio']
			value_ratios[company]['PS'] = fin_ratios[0]['priceToSalesRatio']
			value_ratios[company]['PE'] = fin_ratios[0]['priceEarningsRatio']
			value_ratios[company]['Dividend_Yield'] = fin_ratios[0]['dividendYield']
			value_ratios[company]['Operating_Margin'] = fin_ratios[0]['operatingProfitMargin']
			
			# More financials on growth
			growth_ratios = requests.get(f'https://financialmodelingprep.com/api/v3/financial-growth/{company}?apikey={FMP_API_KEY}').json()
			
			value_ratios[company]['Revenue_Growth'] = growth_ratios[0]['revenueGrowth']
			value_ratios[company]['FCF_Growth'] = growth_ratios[0]['freeCashFlowGrowth']
			value_ratios[company]['Net_Income_Growth'] = growth_ratios[0]['netIncomeGrowth']
			value_ratios[company]['EPS_Growth'] = growth_ratios[0]['epsgrowth']
			value_ratios[company]['Avg_Shares_Dil_Growth'] = growth_ratios[0]['weightedAverageSharesDilutedGrowth']
		
								
	except:
		pass
	
#print(value_ratios)

In [67]:
# Create dataframe of filtered companies with financial stats
df = pd.DataFrame.from_dict(value_ratios, orient='index')
df

,ROE,ROA,Debt_Eq_Ratio,Current_Ratio,PB,PS,PE,Dividend_Yield,Operating_Margin,Revenue_Growth,FCF_Growth,Net_Income_Growth,EPS_Growth,Avg_Shares_Dil_Growth
TSM,0.279990,0.187591,0.492031,1.769721,9.751356,13.467853,34.827524,0.014376,0.423213,0.251638,1.103943,0.463168,0.499411,0.000000
JPM,0.104280,0.008603,11.121076,11.898671,1.418505,3.314832,13.602863,0.032024,0.979288,0.033868,-14.217003,-0.200379,-0.158820,-0.045939
HD,3.899970,0.182287,20.394665,1.229258,84.738245,2.116051,21.727924,0.023076,0.138354,0.198548,0.482662,0.144458,0.169014,0.000929
MA,1.003129,0.190894,4.239712,1.613320,49.539399,20.691870,49.384854,0.005069,0.533495,-0.093704,-0.125956,-0.210273,-0.210046,-0.020548
XOM,-0.136721,-0.067438,1.027356,0.796498,1.166614,1.072252,-8.532814,0.077634,-0.164906,-0.301307,-1.488142,-2.564854,-2.564854,0.000468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SWK,0.111559,0.052354,1.130231,1.324178,2.418532,1.840298,21.679361,0.016846,0.129230,0.006398,0.548566,0.290856,0.236536,0.035222
SWKS,0.195668,0.159555,0.226334,5.168599,5.717152,7.094604,29.218660,0.012895,0.269869,-0.006249,-0.146186,-0.045455,-0.010567,-0.026361
MPLX,-0.055312,-0.019773,1.797419,0.726270,1.865915,2.945502,-33.734182,0.123762,0.406136,-0.043942,0.994026,-1.696999,-1.701646,0.158765
PSX,-0.209387,-0.072641,1.748736,1.394831,1.568268,0.464252,-7.489811,0.052902,-0.023266,-0.402300,-1.865241,-2.292263,-2.332353,-0.032548


# Create Value Filter

In [90]:
# Filter value_candidates based on the specific financial criteria
value_candidates = df[(df['ROE'] >= 0.08) & (df['Debt_Eq_Ratio'] <= 0.50) & (df['Current_Ratio'] >= 1.50) & (df['FCF_Growth'] > 0) & (df['Net_Income_Growth'] > 0) & (df['Operating_Margin'] > 0) & (df['Avg_Shares_Dil_Growth'] <= 0)]
value_candidates.index

Index(['TSM', 'TROW', 'FAST'], dtype='object')

# Filtered Value Company Financial Stats 

In [93]:
research_candidates = df.loc[['TSM','TROW','FAST']]
research_candidates

,ROE,ROA,Debt_Eq_Ratio,Current_Ratio,PB,PS,PE,Dividend_Yield,Operating_Margin,Revenue_Growth,FCF_Growth,Net_Income_Growth,EPS_Growth,Avg_Shares_Dil_Growth
TSM,0.279990,0.187591,0.492031,1.769721,9.751356,13.467853,34.827524,0.014376,0.423213,0.251638,1.103943,0.463168,0.499411,0.000000
TROW,0.307863,0.222601,0.383028,11.055372,4.645468,5.768383,15.089402,0.023624,0.442377,0.104808,0.292997,0.113264,0.142857,-0.031014
FAST,0.314320,0.216687,0.450571,4.079647,9.557687,4.625763,30.407485,0.030754,0.201937,0.058796,0.565823,0.086231,0.087116,-0.002437
